In [1]:
from pprint import pprint
from pathlib import Path
import random

from compgen2 import * 
from compgen2.const import FILENAME_VL

import ipywidgets as widgets
from IPython.display import display
import pandas as pd

In [2]:
vl = pd.DataFrame()
gov = None
data_root = ""

In [5]:
def load_data(btn):
    global vl, gov, data_root
    data_root = Path(data_root_widget.value)
    
    vl = pd.read_parquet(data_root / FILENAME_VL)
    gov = Gov(data_root)
    gov.load_data()
    gov.build_indices()
    
data_root_widget = widgets.Text(
    value='../data',
    placeholder='Pfad zu data_root',
    description='String:',
    disabled=False
)


button = widgets.Button(
    description='Daten laden',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button.on_click(load_data)


widgets.HBox([data_root_widget, button])

In [19]:
def get_matches(b):
    location = pd.Series([location_widget.value])
    
    if prep.value:
        location = Preprocessing.replace_corrections_vl(location)
        location = Preprocessing.replace_characters_vl(location)
        location = Preprocessing.substitute_partial_words(location, data_root)
        location = Preprocessing.substitute_delete_words(location, data_root)
        location = Preprocessing.substitute_full_words(location, data_root)
        
    m=Matcher(gov)
    m.get_match_for_locations(location)
    
    pprint(m.results[location[0]])


location_widget = widgets.Text(
    value=vl.location.sample().values[0],
    disabled=False
)

button = widgets.Button(
    description='Matcher starten',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button.on_click(get_matches)

prep = widgets.Checkbox(
    value=False,
    description='Preprocessing',
    disabled=False,
    indent=False
)


widgets.HBox([location_widget, prep, button])